In [1]:
import sys

In [4]:
import torch
from modelscope import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen3-4B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    torch_dtype="auto",
    device_map="cuda"
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 3/3 [00:03<00:00,  1.24s/it]


In [9]:
# 加载分词器
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [12]:




# 准备输入文本
input_text = "这是一个用来测试的句子。"

inputs = tokenizer(input_text, padding=True, return_tensors='pt')
outputs = model(**inputs)
last_hidden_state = outputs.last_hidden_state # [batch_size, sequence_length, hidden_size]
attention_mask = inputs['attention_mask'] # [batch_size, sequence_length]

RuntimeError: Expected all tensors to be on the same device, but got index is on cpu, different from other tensors on cuda:0 (when checking argument in method wrapper_CUDA__index_select)

In [ ]:


# 准备输入文本
input_text = "这是一个用来测试的句子。"
# 或者一个文本列表
# input_text = ["这是一个用来测试的句子。", "这是第二个句子。"]

# 调用 pipeline 获取 embedding
# pipeline 的输出通常是一个包含 embedding 的列表，如果输入是列表的话
results = text_embedding_pipeline(input_text)

In [2]:
# prepare the model input
prompt = "中文回答 你是什么"
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True # Switches between thinking and non-thinking modes. Default is True.
)


model_inputs = tokenizer([text], return_tensors="pt").to(model.device)# 返回pytorch tensor ，把文字输入转成模型要的输入

# 开始生成
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768
)

output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() # len(model_inputs.input_ids[0]） 是输入的长度，要非输入以外的 则是模型的输出

# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668) # 这里output_ids[::-1] 反过来就是为了找到最后一个think的标识 然后再拿长度减 得到最后一个think的真实index
except ValueError:
    index = 0 #没有说明没有think

thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

print("thinking content:", thinking_content)
print("content:", content)

thinking content: <think>
嗯，用户问“你是什么”，我需要回答这个问题。首先，我需要确定用户想了解什么。可能他们想知道我的身份、功能，或者我的用途。作为通义千问，我是一个大型语言模型，可以回答各种问题，创作文本，进行对话等。

接下来，我需要考虑用户可能的背景。可能是普通用户，想了解我的能力；也可能是开发者，想了解我的技术细节。不过，问题比较简短，可能只是普通用户想确认我的身份。

然后，我需要确保回答准确且简洁。需要说明我是阿里巴巴集团旗下的通义实验室研发的，用于回答问题、创作文本、编程、写作等。同时，可以提到我的特点，比如多语言支持、理解上下文等。

还要注意用户可能的深层需求。比如，他们可能想确认我是否可靠，或者是否有特定功能。所以需要突出我的优势，比如广泛的知识、多语言支持、对话能力等。

最后，检查是否有遗漏的信息，比如是否需要提到伦理准则或应用场景。但问题比较直接，可能不需要过多扩展。保持回答简洁明了，同时涵盖主要功能。
</think>
content: 我是通义千问，阿里巴巴集团旗下的通义实验室研发的大规模语言模型。我能够回答各种问题、创作文本、编程、写作、翻译、数据分析等，支持多语言交流。我的设计目标是提供准确、有用、自然的对话体验，同时遵循伦理准则和法律法规。如果你有任何问题或需要帮助，欢迎随时提问！


In [7]:
output_ids

[151667,
 198,
 106287,
 3837,
 20002,
 56007,
 2073,
 56568,
 102021,
 33590,
 35946,
 85106,
 102104,
 105073,
 1773,
 101140,
 3837,
 35946,
 85106,
 60610,
 20002,
 99172,
 99794,
 99245,
 1773,
 87267,
 99650,
 109623,
 97611,
 101294,
 5373,
 98380,
 3837,
 100631,
 97611,
 105795,
 1773,
 100622,
 31935,
 64559,
 99320,
 56007,
 3837,
 35946,
 101909,
 101951,
 102064,
 104949,
 3837,
 73670,
 102104,
 100646,
 86119,
 3837,
 104223,
 108704,
 3837,
 71817,
 105051,
 49567,
 3407,
 104326,
 3837,
 35946,
 85106,
 101118,
 20002,
 87267,
 9370,
 102193,
 1773,
 104560,
 100714,
 20002,
 3837,
 99172,
 99794,
 97611,
 99788,
 24968,
 74763,
 104560,
 113129,
 3837,
 99172,
 99794,
 97611,
 99361,
 104449,
 1773,
 100632,
 3837,
 86119,
 99792,
 98237,
 99534,
 3837,
 87267,
 100009,
 100714,
 20002,
 99172,
 81167,
 97611,
 101294,
 3407,
 101889,
 3837,
 35946,
 85106,
 103944,
 102104,
 102188,
 100136,
 110485,
 1773,
 85106,
 66394,
 104198,
 107076,
 100338,
 111477,
 31935,
